# cuDF

现在让我们转向一些更高级的 API，从 [cuDF](https://github.com/rapidsai/cudf) 开始。与 `pandas` 类似，`cudf` 库是一个用于处理表格数据集的数据框架包。

数据被加载到 GPU 上，所有操作都使用 GPU 计算执行，但 `cudf` 的 API 对于 `pandas` 用户来说应该非常熟悉。

In [6]:
!pwd

/data/workdir/accelerated-computing-hub/gpu-python-tutorial


In [4]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 25.08 libraries
/bin/sh: line 1: uv: command not found

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files.
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        


In [7]:
import cudf

## 数据加载

在本教程中，我们在 `data/` 目录下存储了一些数据。这些数据大部分都太小，无法真正从 GPU 加速中受益，但让我们还是来探索一下。

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("/data/workdir/accelerated-computing-hub/gpu-python-tutorial/data/pageviews_small.csv", sep=" ")
df.head()

,en.m,Article_51,1,0
0,ja,エレファモン,1,0
1,ang,Flocc:Scīrung,1,0
2,en,Panorama_(La_Dispute_album),1,0
3,fa.m,جاشوا_جکسون,1,0
4,fa.m,خانواده_کندی,2,0


In [10]:
pageviews = cudf.read_csv("/data/workdir/accelerated-computing-hub/gpu-python-tutorial/data/pageviews_small.csv", sep=" ")
pageviews.head()

,en.m,Article_51,1,0
0,ja,エレファモン,1,0
1,ang,Flocc:Scīrung,1,0
2,en,Panorama_(La_Dispute_album),1,0
3,fa.m,جاشوا_جکسون,1,0
4,fa.m,خانواده_کندی,2,0


这个 `pageviews.csv` 文件包含了来自各种语言的维基百科的超过 `1M` 条页面浏览量记录。

让我们重命名列并删除未使用的 `x` 列。

In [11]:
pageviews.columns = ['project', 'page', 'requests', 'x']

pageviews = pageviews.drop('x', axis=1)

pageviews

,project,page,requests
0,ja,エレファモン,1
1,ang,Flocc:Scīrung,1
2,en,Panorama_(La_Dispute_album),1
3,fa.m,جاشوا_جکسون,1
4,fa.m,خانواده_کندی,2
...,...,...,...
1118994,ja,STUTS,2
1118995,ml.m,മാർച്ച്_18,3
1118996,es.m,Pablo_Sánchez_Velarde,1
1118997,ar,مترو_الجزائر_العاصمة,1


接下来，让我们统计这个数据集中有多少条英文记录。

In [12]:
print(pageviews[pageviews.project == 'en'].count())

project     196882
page        196881
requests    196882
dtype: int64


然后让我们执行一个分组操作，按语言统计所有页面。

In [13]:
grouped_pageviews = pageviews.groupby('project').count().reset_index()
grouped_pageviews

,project,page,requests
0,hu.m.b,21,21
1,en.voy,564,564
2,ba.m,3,3
3,es.v,30,30
4,tr.b,13,13
...,...,...,...
1347,pi,12,12
1348,pt.q,23,23
1349,id.m.d,393,393
1350,dty.m,1,1


最后，让我们查看英语、法语、中文和波兰语的结果。

In [14]:
print(grouped_pageviews[grouped_pageviews.project.isin(['en', 'fr', 'zh', 'pl'])])

     project    page  requests
257       en  196881    196882
781       zh   17577     17577
940       pl   11931     11931
1077      fr   33915     33915


如果你之前使用过 `pandas`，那么所有这些语法对你来说应该非常熟悉。就像 `cupy` 实现了 `numpy` API 的大部分功能一样，`cudf` 实现了 `pandas` API 的大部分功能。

唯一的区别是，我们所有的过滤和分组操作都在 GPU 上执行，而不是在 CPU 上，从而获得更好的性能。

### 字符串

GPU 历来以数值计算而闻名，并没有被用于处理更复杂的对象。使用 cuDF，字符串操作也可以通过专门的内核进行加速。

这意味着像大写字符串这样的操作可以在 GPU 上并行化。

In [15]:
pageviews[pageviews.project == 'en'].page.str.upper()

2                    PANORAMA_(LA_DISPUTE_ALBUM)
9                                 OPERATING_COST
19         RICHARD_BARLOW_(INTELLIGENCE_ANALYST)
27                 ST_VINCENT_AND_THE_GRENADINES
28                                     IAN_MCKEE
                           ...                  
1118958                               DEVIL_GIRL
1118969           PHILOSOPHERS_LEGACY_(HEIRLOOM)
1118973                               KINGS_HILL
1118986                   TIONNE_"T-BOZ"_WATKINS
1118989                             ANTINEUTRINO
Name: page, Length: 196882, dtype: object

In [16]:
pageviews_en = pageviews[pageviews.project == 'en']
print(pageviews_en.page.str.upper().head())

2               PANORAMA_(LA_DISPUTE_ALBUM)
9                            OPERATING_COST
19    RICHARD_BARLOW_(INTELLIGENCE_ANALYST)
27            ST_VINCENT_AND_THE_GRENADINES
28                                IAN_MCKEE
Name: page, dtype: object


### 用户自定义函数（UDF）

cuDF 还支持用户自定义函数（UDF），可以在 GPU 上并行地映射到 Series 或 DataFrame 上。

UDF 可以定义为接受单个值的纯 Python 函数。当我们调用 `.apply()` 时，这些函数将在运行时由 Numba 编译成可以在 GPU 上运行的代码。

In [17]:
def udf(x):
    if x < 5:
        return 0
    return x

In [18]:
pageviews.requests = pageviews.requests.apply(udf)

ValueError: user defined function compilation failed.

In [19]:
pageviews.requests

0          1
1          1
2          1
3          1
4          2
          ..
1118994    2
1118995    3
1118996    1
1118997    1
1118998    1
Name: requests, Length: 1118999, dtype: int64

也可以直接使用 Numba 编写内核，这些内核接受指向输入列和输出列的指针以及额外的参数。然后，内核可以使用 `cuda.grid`，就像我们在第 2/3 章中所做的那样，来获取要操作的索引。

然后我们使用 `.forall()` 将我们的内核映射到列上。

In [ ]:
pageviews['mul_requests'] = 0.0

In [ ]:
from numba import cuda
from numba import config as numba_config
numba_config.CUDA_ENABLE_PYNVJITLINK = True

In [ ]:
@cuda.jit
def multiply(in_col, out_col, multiplier):
    i = cuda.grid(1)
    if i < in_col.size: # boundary guard
        out_col[i] = in_col[i] * multiplier

In [ ]:
multiply.forall(len(pageviews))(pageviews['requests'], pageviews['mul_requests'], 10.0)

In [ ]:
print(pageviews.head())

## 滚动窗口

在 cuDF 中还支持在滚动窗口上应用内核。这实际上是一个一维模板，可以让我们基于相邻元素执行操作。

![](images/rolling-windows.png)

In [ ]:
def neigborhood_mean(window):
    c = 0
    for val in window:
        c += val
    return c / len(window)

In [ ]:
pageviews.requests.rolling(3, 1, True).apply(neigborhood_mean)